## Análise de dados da fila da creche no município de São Paulo

Obtendo os dados das matrículas na rede municipal de educação, obtidos [aqui](http://dados.prefeitura.sp.gov.br/dataset/demanda-por-vagas-educacao-infantil-e-eja):

In [32]:
import pandas as pd

data_creche = pd.read_csv('dados/evolucaodemanda.csv', sep = ';')

Temos que filtrar apenas as colunas que necessárias. Além disso, é preciso limpar os dados, pois algumas linhas possuem os números com um ponto (.) para indicar o milhar. Por conveniência, o ano de referência será deixado em uma coluna a parte. 

In [33]:
for i in data_creche.index:
    if not data_creche.at[i, 'MAT_CRECHE'].is_integer():
        data_creche.at[i, 'MAT_CRECHE'] *= 1000
    if not data_creche.at[i, 'DEM_CRECHE'].is_integer():
        data_creche.at[i, 'DEM_CRECHE'] *= 1000
    data_creche.at[i, 'MES'] = data_creche.at[i, 'MES_ANO_REF'][:3]
    data_creche.at[i, 'ANO'] = int(data_creche.at[i, 'MES_ANO_REF'][4:]) + 2000
    
data_creche = data_creche[['DISTRITO', 'MAT_CRECHE', 'DEM_CRECHE', 'MES', 'ANO']]

Agora, vamos obter os dados populacionais por distrito, com os dados da [SEADE](http://produtos.seade.gov.br/produtos/projpop/):

In [34]:
data_pop = pd.read_csv('dados/populacao.csv', sep = ';', encoding = "ISO-8859-1")

Para que os dados populacionais fiquem mais próximos aos da creche, vamos alterar o nome do distrito, deixando-o sem acentos e com caixa alta. Como os dados da população estão em números inteiros, não precisaremos remover os eventuais pontos (.) que indicam o milhar. Além disso, as colunas que selecionaremos terão os mesmos nomes nas nossas duas bases de dados (creche e população).

In [35]:
# Filtra as colunas
data_pop = data_pop[['Localidades', 'Períodos', 'População de 0 a 3 Anos']]
data_pop.columns = ['DISTRITO', 'ANO', 'TOTAL']
    
# Tira os acentos dos nomes dos distritos e deixa o nome de cada distrito em caixa alta e 
data_pop['DISTRITO'] = data_pop['DISTRITO'].str.normalize('NFKD')
data_pop['DISTRITO'] = data_pop['DISTRITO'].str.encode('ascii', errors='ignore').str.decode('utf-8')
data_pop['DISTRITO'] = data_pop['DISTRITO'].str.upper()

Precisamos juntar os dados, para que a análise seja feita de maneira mais eficiente. Teremos um dicionário com os seguintes valores:

- Nome do distrito (em caixa alta e sem acentos);
- Número de matriculas em creches;
- Demanda de vagas em creches;
- Mês de referência (mmm);
- Ano de referência (aaaa);
- População do distrito na faixa etária 00 a 03 anos.

In [36]:
data_creche_pop = data_creche.merge(data_pop, on = ['DISTRITO', 'ANO'])